In [1]:
import pandas as pd
import urllib.request as ur
from datetime import date, timedelta, datetime
import os

In [2]:
### ### ### CHANGE WORKING DIRECTORY TO OUR PIPELINE STORAGE ### ### ### 
print('Old Working Directory:',os.getcwd())
os.chdir('/media/tom/Data2/Alpha Hound/Equity Algo/Exchange FTP')
print('New Working Directory:',os.getcwd())

Old Working Directory: /home/tom/Python
New Working Directory: /media/tom/Data2/Alpha Hound/Equity Algo/Exchange FTP


In [3]:
today = date.today()
#nasdaq_date = today.strftime('%Y%m%d')
#nyse_date = today.strftime('%d-%b-%Y')

In [4]:
##generator function below taken from stack overflow https://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
#generate a range of datetimes to iterate over for the data fetch
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [5]:
df = pd.read_csv('threshold_securities.csv')
#df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')
last_date = df['date'].iloc[-1]
last_date = datetime.strptime(last_date,'%Y-%m-%d')


/home/tom/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
start_date = last_date.date()
end_date = today + timedelta(days=1)

In [7]:
nasdaq_fails = []
nyse_fails = []

for single_date in daterange(start_date, end_date):
    try:
        query_date = single_date.strftime("%Y%m%d")
        url = 'ftp://ftp.nasdaqtrader.com/symboldirectory/regsho/nasdaqth{0}.txt'.format(query_date)
        req = ur.Request(url)
        data = ur.urlopen(req)
        data = pd.read_csv(data,sep="|")
        data.drop(data.tail(1).index,inplace=True)
        #print(single_date)
        data['date'] = single_date
        data['exchange'] = 'NASDAQ'
        #print(data)
        df = df.append(data,ignore_index=True)
    except:
        nasdaq_fails.append(single_date)
    try:
        query_date = single_date.strftime('%d-%b-%Y')
        url = 'https://www.nyse.com/api/regulatory/threshold-securities/download?selectedDate={0}&market='.format(query_date)
        req = ur.Request(url)
        data = ur.urlopen(req)
        data = pd.read_csv(data,sep="|")
        data.drop(data.tail(1).index,inplace=True)
        #print(single_date)
        data['date'] = single_date
        data['exchange'] = 'NYSE'
        #print(data)
        df = df.append(data,ignore_index=True)
    except:
        nyse_fails.append(single_date)

In [8]:
#df = df.sort_values(by='date')


,Symbol,Security Name,Market Category,Reg SHO Threshold Flag,Filler,Filler.1,date,exchange,Rule 3210,Filler
0,AAII,AAIPHARMA INC,Q,Y,NaN,NaN,2005-01-07,NASDAQ,NaN,NaN
1,PPYR,PRIMEPLAYER INC,u,Y,NaN,NaN,2005-01-07,NASDAQ,NaN,NaN
2,PPHD,PREMIER PLATFORMS HLDG INC NEW,u,Y,NaN,NaN,2005-01-07,NASDAQ,NaN,NaN
3,NMCX,NMC INC,u,Y,NaN,NaN,2005-01-07,NASDAQ,NaN,NaN
4,NLRRF,NELSON RESOURCES LTD,u,Y,NaN,NaN,2005-01-07,NASDAQ,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
469214,SOXS,Direxion Daily Semiconductor Bear 3X Shares,NYSE Arca,Y,NaN,NaN,2020-10-16,NYSE,NaN,NaN
469215,TDVG,T. Rowe Price Dividend Growth ETF,NYSE Arca,Y,NaN,NaN,2020-10-16,NYSE,NaN,NaN
469216,TMF,Direxion Daily 20+ Year Treasury Bull 3X Share...,NYSE Arca,Y,NaN,NaN,2020-10-16,NYSE,NaN,NaN
469217,WRBPRB,W.R. BERKLEY CORPORATION 5.625% Subordinated D...,NYSE,Y,NaN,NaN,2020-10-16,NYSE,NaN,NaN


In [9]:
df.to_csv('threshold_securities.csv',index=False)

In [10]:
print(len(nasdaq_fails),nasdaq_fails)
print(len(nyse_fails),nyse_fails)

2 [datetime.date(2020, 10, 17), datetime.date(2020, 10, 18)]
0 []
